# Self Study 8: The Kaldor Model

Let's explore the Kaldor model we discussed last week in the lecture. This time, we will use different functional forms for Savings and Investment than the ones you saw in class.

The objective of using these functions is to learn how to explore a nonlinear model using the tools we have already encountered, in order to make sense of a model where analytical solutions may not exist.

Note: The the takeaway from this exercise is not to memorize specific functions, but to understand the methodology for exploring the model:
1. Plot the functions to understand their behavior 
2. Find fixed points visually
3. Look at the Phase Diagram to understand the dynamics of the system
4. What if we assume a simpler functional form? How does the model change? Do the conclusions change?


## The Model

Our economy is described by two variables:

1. **Output ($ Y $)**: The total value of all goods and services produced.
2. **Capital Stock ($ K $)**: The total capital.

Their dynamics is governed by the following ODEs:

$$
\frac{dY}{dt} = \alpha \left( I(Y, K) - S(Y, K) \right)
$$
$$
\frac{dK}{dt} = I(Y, K) - \delta K
$$

where:
- $ \alpha $ is the speed at which Y changes in response to discrepancies in investment and savings.
- $ \delta $ is capital depreciation rate.


### Investment
For our exercise, the investment function $ I(Y, K) $ is given by logistic function of $Y$ and a linear function of $K$:

$$
I(Y, K) = 3 + \frac{1}{1+\exp(-2.5\cdot(Y-4))} - \frac{K}{2}
$$

This equations satisfies the necessary properties of the investment function:

- $I$ increases in $ Y $ -> $\partial I/\partial Y > 0$
- $I$ decreases in $ K $ -> $\partial I/\partial K < 0$

### Savings
The savings function $ S(Y, K) $ must be positive and increasing in $ Y $. For our exercise, we assume that it only depends on $Y$ and not on $K$, and we will use the following function:

$$
S(Y, K) = sY
$$

where $ s $ is the marginal propensity to save, indicating the fraction of output that is saved.

## Section 1: Basic Dynamics

### Task 1: Define functions

- $S$
- $I$
- $\frac{dY}{dt}$
- $\frac{dK}{dt}$

In [1]:
import numpy as np

def investment(Y, K):
    # your code here
    Y_term = 1/(1+np.exp(-2.5*(Y-4)))
    K_term = K/2
    return 3 + Y_term - K_term
    # return 3 + Y/3 - K_term

def savings(Y, s):
    # your code here
    return s*Y


def dYdt(I, S, alpha):
    # your code here
    return alpha * (I-S)

def dKdt(K, I, delta):
    # your code here
    return I - delta * K

### Task 2: S/I curves

- Plot the savings and investment curves for $Y$ for different values of $K$.
- Also, explore parameters for $s$ to see how they affect the number of equilibria. 

Use `ipywidgets` to create interactive plots.

In [2]:
import matplotlib.pyplot as plt
from ipywidgets import interact

Y = np.linspace(1,7,100)
@interact(s=(0.1, 1, 0.1), kk=(0.1, 10, 0.1))
def plot_si(s=0.3, kk=4.6):
    # your code here
    savings_vals = [savings(Y=y, s=s) for y in Y]
    investment_vals = [investment(Y=y, K=kk) for y in Y]
    plt.figure(figsize=(6,4))
    plt.plot(Y, savings_vals, label='S')
    plt.plot(Y, investment_vals, label='I')
    plt.grid(True)
    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=0.3, description='s', max=1.0, min=0.1), FloatSlider(value=4.6, descri…

- **Questions:** When changing $K$ and $s$ what do you observe? What is the economic interpretation?
- **Answer:** Increase in $K$ gives an immediate decrease to $I$ as expected since a high amount of capital in the economy deincentivizes investments, and vice versa. Increase in $s$ gives an increase in the total Savings $S$ and steepens the slope as the line is just this parameter multiplied with the x-value. Economically this means that the bigger the savings rate, the more savings in the whole economy.

### Task 3: Dynamics of Y

Plot:
- $\dot{Y}$ vs $Y$ for different values of $K$ and $s$.
- $\dot{K}$ vs $K$ for different values of $Y$, $s$, and $\delta$.

This plot will help you understand the dynamics of the system without having to solve the ODEs.

Note: $\dot{Y}$ is just another way of writing $\frac{dY}{dt}$.

In [3]:
import matplotlib.pyplot as plt
from ipywidgets import interact
Y2 = np.linspace(1,10,100)

@interact(s=(0.1, 1, 0.01), kk=(0.1, 6, 0.1), alpha=(0.01, 3.0, 0.01), delta=(0.01, 1, .01))
def plot_change_y(s, kk, alpha, delta):
    # your code here
    changes_in_y = [dYdt(I=investment(Y=y, K=kk), S=savings(Y=y, s=s), alpha=alpha) for y in Y2]
    changes_in_k = [dKdt(K=kk, I=investment(Y=y, K=kk), delta=delta) for y in Y2]

    plt.figure(figsize=(6,4))
    plt.plot(Y2, changes_in_y, label='dYdt')
    plt.plot(Y2, changes_in_k, label='dKdt')
    plt.grid(True)
    plt.legend()
    plt.show()

interactive(children=(FloatSlider(value=0.55, description='s', max=1.0, min=0.1, step=0.01), FloatSlider(value…

- **Question:** What are the points at which $\dot{Y} = 0$ and $\dot{K} = 0$? What is the dynamics of $Y$ around these points?
- **Answer:**

- **Question:** What happens to the fixed points when $\alpha$ changes? What does it mean for the dynamics of the system?
- **Answer:** Fixed points stay at the same place when $\alpha$ changes!

## Section 2: Phase Diagram

Plot the phase diagram for the system.

In [4]:
def kaldor_system(YK, t, s, alpha, delta):
    # your code here
    Y, K = YK
    dydt = dYdt(I=investment(Y=Y, K=K), S=savings(Y=Y, s=s), alpha=alpha)
    dkdt = dKdt(K=K, I=investment(Y=Y,K=K), delta=delta)
    return dydt, dkdt

In [8]:
from scipy.integrate import odeint
from ipywidgets import fixed

def solve_and_plot(alpha, delta, s):
    t = np.linspace(0, 200, 400)
    u0 = [5, 4.4]
    sol = odeint(kaldor_system, u0, t, args=(s, alpha, delta))
    fig, ax = plt.subplots(1, 2, figsize=(16, 8))
    ax[0].plot(t, sol[:, 0], label='Output')
    ax[0].plot(t, sol[:, 1], label='Capital')
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('Output / Capital')
    
    ax[1].plot(sol[:, 0], sol[:, 1], label='Output', color='red')
    ax[1].set_xlabel('Output')
    ax[1].set_ylabel('Capital')
    
    # plot origin
    ax[1].plot(*u0, 'ko')
    
    # streamplot
    Y, K = np.meshgrid(np.linspace(0, 8, 30), np.linspace(0, 8, 30))
    dYdt, dKdt = kaldor_system((Y, K), t=None, s=s, alpha=alpha, delta=delta)
    ax[1].streamplot(Y, K, dYdt, dKdt, color='gray', density=2)
    ax[1].set_xlabel('Output')
    ax[1].set_ylabel('Capital')
    

In [9]:
# Use these parameters to start, watch at what happens at delta=0.2
interact(solve_and_plot, alpha=fixed(2.2), delta=(0.01, 0.3, 0.01), s=fixed(0.25));


interactive(children=(FloatSlider(value=0.15000000000000002, description='delta', max=0.3, min=0.01, step=0.01…

## Section 3: What if $I$ is linear also in $Y$?

Rerun the above code but with the following investment function:
$$
I(Y, K) = 3 + \frac{Y}{3} - \frac{K}{2}
$$

- **Question:** What are the point(s) at which $\dot{Y} = 0$ and $\dot{K} = 0$? What is the dynamics of $Y$ around these points?
- **Answer:**

- **Question:** If we find that $Y$ has multiple solutions in one case but not the other, what economic meaning does this have?
- **Answer:**